In [4]:
# train a Deep Learning model on MNIST Data to recognize digits in images using pytorch
# using the context manager

In [5]:
# import libraries
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

from codecarbon import track_emissions

In [8]:
@track_emissions(project_name="mnist")

def train_model():
    # Download training data from open datasets.
    training_data = datasets.MNIST(
        root="data",
        train=True,
        download=True,
        transform=ToTensor(),
    )

    # Download test data from open datasets.
    test_data = datasets.MNIST(
        root="data",
        train=False,
        download=True,
        transform=ToTensor(),
    )

    # set the batch size
    batch_size = 64

    # Create data loaders.
    train_dataloader = DataLoader(training_data, batch_size=batch_size)
    test_dataloader = DataLoader(test_data, batch_size=batch_size)

    for X, y in test_dataloader:
        print(f"Shape of X [N, C, H, W]: {X.shape}")
        print(f"Shape of y: {y.shape} {y.dtype}")
        break

    # creating model
    # Get cpu, gpu or mps device for training.
    device = (
        "cuda"
        if torch.cuda.is_available()
        else "mps"
        if torch.backends.mps.is_available()
        else "cpu"
    )
    print(f"Using {device} device")

    # Define model
    class NeuralNetwork(nn.Module):
        def __init__(self):
            super().__init__()
            self.flatten = nn.Flatten()
            self.linear_relu_stack = nn.Sequential(
                nn.Linear(28*28, 512),
                nn.ReLU(),
                nn.Linear(512, 512),
                nn.ReLU(),
                nn.Linear(512, 10)
            )

        def forward(self, x):
            x = self.flatten(x)
            logits = self.linear_relu_stack(x)
            return logits

    model = NeuralNetwork().to(device)
    print(model)

    # optimizer and loss function
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

    def train(dataloader, model, loss_fn, optimizer):
        size = len(dataloader.dataset)
        model.train()
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)

            # Compute prediction error
            pred = model(X)
            loss = loss_fn(pred, y)

            # Backpropagation
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            if batch % 100 == 0:
                loss, current = loss.item(), (batch + 1) * len(X)
                print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

    def test(dataloader, model, loss_fn):
        size = len(dataloader.dataset)
        num_batches = len(dataloader)
        model.eval()
        test_loss, correct = 0, 0
        with torch.no_grad():
            for X, y in dataloader:
                X, y = X.to(device), y.to(device)
                pred = model(X)
                test_loss += loss_fn(pred, y).item()
                correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        test_loss /= num_batches
        correct /= size
        print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

    epochs = 10
    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train(train_dataloader, model, loss_fn, optimizer)
        test(test_dataloader, model, loss_fn)
    print("Done!")

    return model

if __name__ == "__main__":
    train_model()

[codecarbon INFO @ 23:23:40] [setup] RAM Tracking...
[codecarbon INFO @ 23:23:40] [setup] GPU Tracking...
[codecarbon INFO @ 23:23:40] No GPU found.
[codecarbon INFO @ 23:23:40] [setup] CPU Tracking...
[codecarbon WARNING @ 23:23:40] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 23:23:41] CPU Model on constant consumption mode: Apple M1
[codecarbon INFO @ 23:23:41] >>> Tracker's metadata:
[codecarbon INFO @ 23:23:41]   Platform system: macOS-14.0-arm64-arm-64bit
[codecarbon INFO @ 23:23:41]   Python version: 3.11.5
[codecarbon INFO @ 23:23:41]   CodeCarbon version: 2.3.1
[codecarbon INFO @ 23:23:41]   Available RAM : 16.000 GB
[codecarbon INFO @ 23:23:41]   CPU count: 8
[codecarbon INFO @ 23:23:41]   CPU model: Apple M1
[codecarbon INFO @ 23:23:41]   GPU count: None
[codecarbon INFO @ 23:23:41]   GPU model: None


Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64
Using mps device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)
Epoch 1
-------------------------------
loss: 2.307909  [   64/60000]
loss: 2.296436  [ 6464/60000]
loss: 2.297916  [12864/60000]
loss: 2.284069  [19264/60000]
loss: 2.295702  [25664/60000]
loss: 2.286726  [32064/60000]
loss: 2.270726  [38464/60000]
loss: 2.279179  [44864/60000]
loss: 2.266067  [51264/60000]
loss: 2.252872  [57664/60000]
Test Error: 
 Accuracy: 38.8%, Avg loss: 2.256104 

Epoch 2
-------------------------------
loss: 2.260781  [   64/60000]
loss: 2.249495  [ 6464/60000]
loss: 2.256535  [12864/60000]
loss: 2.220991  [19264/60000]
loss: 2.239295  [2

[codecarbon INFO @ 23:23:59] Energy consumed for RAM : 0.000025 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 23:23:59] Energy consumed for all CPUs : 0.000021 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 23:23:59] 0.000046 kWh of electricity used since the beginning.


loss: 2.151026  [44864/60000]
loss: 2.097886  [51264/60000]
loss: 2.076109  [57664/60000]
Test Error: 
 Accuracy: 62.4%, Avg loss: 2.072435 

Epoch 4
-------------------------------
loss: 2.076640  [   64/60000]
loss: 2.053730  [ 6464/60000]
loss: 2.085165  [12864/60000]
loss: 1.958799  [19264/60000]
loss: 1.999657  [25664/60000]
loss: 2.004137  [32064/60000]
loss: 1.960239  [38464/60000]
loss: 2.005347  [44864/60000]
loss: 1.912456  [51264/60000]
loss: 1.878706  [57664/60000]
Test Error: 
 Accuracy: 66.4%, Avg loss: 1.869066 

Epoch 5
-------------------------------
loss: 1.876691  [   64/60000]
loss: 1.833427  [ 6464/60000]
loss: 1.890536  [12864/60000]
loss: 1.694905  [19264/60000]
loss: 1.747511  [25664/60000]
loss: 1.751159  [32064/60000]
loss: 1.693669  [38464/60000]
loss: 1.774016  [44864/60000]
loss: 1.634020  [51264/60000]
loss: 1.588840  [57664/60000]
Test Error: 
 Accuracy: 70.7%, Avg loss: 1.571565 

Epoch 6
-------------------------------
loss: 1.591898  [   64/60000]
loss

[codecarbon INFO @ 23:24:14] Energy consumed for RAM : 0.000050 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 23:24:14] Energy consumed for all CPUs : 0.000042 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 23:24:14] 0.000092 kWh of electricity used since the beginning.


Test Error: 
 Accuracy: 74.5%, Avg loss: 1.256369 

Epoch 7
-------------------------------
loss: 1.300146  [   64/60000]
loss: 1.201581  [ 6464/60000]
loss: 1.299737  [12864/60000]
loss: 1.098346  [19264/60000]
loss: 1.124457  [25664/60000]
loss: 1.117518  [32064/60000]
loss: 1.071656  [38464/60000]
loss: 1.231601  [44864/60000]
loss: 1.086040  [51264/60000]
loss: 1.032810  [57664/60000]
Test Error: 
 Accuracy: 78.0%, Avg loss: 1.011390 

Epoch 8
-------------------------------
loss: 1.079807  [   64/60000]
loss: 0.969918  [ 6464/60000]
loss: 1.057873  [12864/60000]
loss: 0.902882  [19264/60000]
loss: 0.914954  [25664/60000]
loss: 0.906845  [32064/60000]
loss: 0.864732  [38464/60000]
loss: 1.035041  [44864/60000]
loss: 0.920325  [51264/60000]
loss: 0.869551  [57664/60000]
Test Error: 
 Accuracy: 80.8%, Avg loss: 0.845299 

Epoch 9
-------------------------------
loss: 0.930120  [   64/60000]
loss: 0.814726  [ 6464/60000]
loss: 0.886756  [12864/60000]
loss: 0.773881  [19264/60000]
loss

[codecarbon INFO @ 23:24:29] Energy consumed for RAM : 0.000075 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 23:24:29] Energy consumed for all CPUs : 0.000063 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 23:24:29] 0.000138 kWh of electricity used since the beginning.


loss: 0.671033  [32064/60000]
loss: 0.622892  [38464/60000]
loss: 0.797062  [44864/60000]
loss: 0.725193  [51264/60000]
loss: 0.692374  [57664/60000]


[codecarbon INFO @ 23:24:31] 
Graceful stopping: collecting and writing information.
Please wait a few seconds...
[codecarbon INFO @ 23:24:31] Energy consumed for RAM : 0.000079 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 23:24:31] Energy consumed for all CPUs : 0.000066 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 23:24:31] 0.000144 kWh of electricity used since the beginning.
[codecarbon INFO @ 23:24:31] Done!



Test Error: 
 Accuracy: 83.8%, Avg loss: 0.653178 

Done!
